In [1]:
%load_ext autoreload
%autoreload

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard
from keras import metrics
import json
from helper_functions import import_data, neuralnet_model, batch_generator
from sklearn.metrics import confusion_matrix
from os import listdir

Using TensorFlow backend.


In [3]:
X_test, y_test = import_data('jsons_test/' + listdir('jsons_test')[0])

In [6]:
batch_size = 2000
nb_epoch = 10
samples_per_epoch = 15

for x_, y_ in batch_generator('json_test', batch_size, samples_per_epoch):
    X_debug = x_
    y_debug = y_
    break

In [8]:
y_debug

array([[1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
    

In [4]:
autoencoder_model_created = False
model_path = 'models/basic_autoencoder1.h5'

if not autoencoder_model_created:
    model = neuralnet_model(X_test)

    batch_size = 2000
    nb_epoch = 10
    samples_per_epoch = 15

    # instantiate callbacks
    tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=2)

    # try different number of epochs - 10 gives good performanace 
    """model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
              validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


    model.fit_generator(generator=batch_generator('jsons_full', batch_size, samples_per_epoch),
                        epochs=nb_epoch,
                        steps_per_epoch=samples_per_epoch)


    scores = model.evaluate(X_test, y_test)
    print('Test mse = {}'.format(scores[0]))

    y_predictions = model.predict(X_test)
    
    model.save(model_path)

else:
    model = load_model(model_path)
    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
15/15 [==============================] - 7s 489ms/step - loss: 1.3297 - acc: 0.5008
Epoch 2/10
15/15 [==============================] - 7s 455ms/step - loss: 0.7472 - acc: 0.5367
Epoch 3/10
15/15 [==============================] - 7s 455ms/step - loss: 0.4396 - acc: 0.8254
Epoch 4/10
15/15 [==============================] - 7s 450ms/step - loss: 0.3045 - acc: 0.8891
Epoch 5/10
15/15 [==============================] - 7s 448ms/step - loss: 0.2654 - acc: 0.8979
Epoch 6/10
15/15 [==============================] - 7s 455ms/step - loss: 0.2459 - acc: 0.9050
Epoch 7/10
15/15 [==============================] - 7s 451ms/step - loss: 0.2493 - acc: 0.9053
Epoch 8/10
15/15 [==============================] - 7s 455ms/step - loss: 0.2298 - acc: 0.9082
Epoch 9/10
15/15 [==============================] - 7s 454ms/step - loss: 0.2301 - acc: 0.9062
Epoch 10/10
4683/4683 [==

In [27]:
y_predictions = X_test_decoded

In [34]:
confusions = confusion_matrix(y_test, y_predictions > 0.5)

In [36]:
confusions

array([[ 528,   39],
       [ 461, 3655]])

In [33]:
y_predictions < 0.5

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [24]:
y_predictions.astype(int)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [ ]:
X_test_decoded[X_test_decoded < 0.5].shape

In [16]:
y_test.shape

(4683,)

In [18]:
y_test[y_test == 1].shape

(4116,)

In [19]:
X_test_decoded

array([[0.999895  ],
       [0.9999863 ],
       [0.99558365],
       ...,
       [0.9860421 ],
       [0.9999702 ],
       [0.99895084]], dtype=float32)